In [2]:
import pandas as pd
import duckdb
from fila_estacoes_utils import P_1
ouro_conn = duckdb.connect('ouro_db')

In [5]:
ouro_conn.execute("SHOW TABLES").fetch_df()

,name
0,dim_estacoes
1,fato_estacoes_distancia
2,fato_estacoes_intersecao
3,fato_estacoes_latlon_produtos_df
